# Train

In [2]:
%reload_ext autoreload
%autoreload 2

from langlearn.dataprep.utils import read_cita_tsv

data_prefix = './langlearn/data'
training_cita_tsv = data_prefix+'/raw/LangLearn_Training_Data/CItA/Training_CItA.tsv'

df,tsv_train = read_cita_tsv(training_cita_tsv)
df

,author,essay,sequence_abs
0,A_3375,4355,0
1,A_3375,3375,1
2,A_3375,4466,2
3,A_3375,5217,3
4,A_3375,5310,4
...,...,...,...
834,A_4314,4613,7
835,A_4314,4314,8
836,A_4131,4216,7
837,A_4131,4131,9


In [3]:
from collections import defaultdict

import numpy as np
import pandas as pd


dtypes = defaultdict(np.float32, fn='str', predicted='category')

training_cita_malt = data_prefix + '/interim/LangLearn_Training_Data/CItA/malt.csv'

df_malt = pd.read_csv(training_cita_malt, sep=',', dtype=dtypes, keep_default_na=False)
df_malt['predicted'].replace(['B1', 'B2', 'C1', 'C2'], [1,2,3,4], inplace=True)
df_malt['predicted'] = df_malt['predicted'].astype(pd.CategoricalDtype(ordered=True))
df_malt.rename(columns={"fn":"essay"}, inplace=True)
df_malt['essay'] = df_malt['essay'].str.replace('.txt','')
df_malt.sort_values('essay', inplace=True, ignore_index=True)

from sklearn.feature_selection import VarianceThreshold
df_malt_vt = VarianceThreshold().set_output(transform='pandas')
df_malt_numbers = df_malt.select_dtypes('number').copy()
df_malt.drop(columns=df_malt_numbers.columns, inplace=True)
df_malt = pd.concat([df_malt, df_malt_vt.fit_transform(df_malt_numbers)], axis=1)

print(df_malt[df_malt.isna().any(axis=1)])
print(df_malt[df_malt.isnull().any(axis=1)])
print("#")
print(df_malt)

Empty DataFrame
Columns: [essay, predicted, probability_B1, probability_B2, probability_C1, probability_C2, scalarFeatures_Depth of the Parse Trees_0, scalarFeatures_Lexical Diversity_0, scalarFeatures_Maximal Non-Verbal Phrase - Mean_0, scalarFeatures_Maximal Non-Verbal Phrase - Std.Dev._0, scalarFeatures_Mean Length of Clauses (Tokens) - Mean_0, scalarFeatures_Non-Verbal Chains Length - Mean_0, scalarFeatures_Non-Verbal Chains Length - Std.Dev._0, scalarFeatures_Referential Cohesion - Mean_0, scalarFeatures_Referential Cohesion - Std.Dev._0, scalarFeatures_Sentence Length (in Tokens) - Mean_0, scalarFeatures_Sentence Length (in Tokens) - Std.Dev._0, scalarFeatures_Subordinate Ratio - Mean_0, scalarFeatures_Subordinate Ratio - Std.Dev._0, scalarFeatures_Text Length (in Lemmas)_0, scalarFeatures_Text Length (in Sentences)_0, scalarFeatures_Token Length - Mean_0, scalarFeatures_Token Length - Std.Dev._0, scalarFeatures_Verbal Roots_0, simpleScalarFeatures_Lunghezza del testo_0, simpleSc

In [94]:
# spaCy - IT

import spacy
from spacy.lang.it.examples import sentences 

nlp = spacy.load("it_core_news_lg")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.morph, token.dep_)

Apple vuole comprare una startup del Regno Unito per un miliardo di dollari
Apple Apple PROPN  nsubj
vuole volere AUX Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin aux
comprare comprare VERB VerbForm=Inf ROOT
una uno DET Definite=Ind|Gender=Fem|Number=Sing|PronType=Art det
startup startup NOUN Gender=Fem|Number=Sing obj
del di il ADP Definite=Def|Gender=Masc|Number=Sing|PronType=Art case
Regno Regno PROPN  nmod
Unito Unito PROPN  flat:name
per per ADP  case
un uno DET Definite=Ind|Gender=Masc|Number=Sing|PronType=Art det
miliardo miliardo NOUN Gender=Masc|Number=Sing obl
di di ADP  case
dollari dollaro NOUN Gender=Masc|Number=Plur nmod


In [4]:
import spacy
nlp = spacy.load("it_core_news_lg")

import langlearn.dataprep.utils
training_cita_essays_fn = data_prefix + '/interim/LangLearn_Training_Data/CItA/Essays_CItA.xml'
txts = langlearn.dataprep.utils.get_txts(training_cita_essays_fn, shrink_whitespaces=True)
docs = dict(zip(list(txts.keys()), list(nlp.pipe(list(txts.values())))))

testing_cita_essays_fn = data_prefix + '/interim/LangLearn_Test_Data/CItA/Essays_Test_CItA.xml'
txts_test = langlearn.dataprep.utils.get_txts(testing_cita_essays_fn, shrink_whitespaces=True)
docs.update(dict(zip(list(txts_test.keys()), list(nlp.pipe(list(txts_test.values()))))))

#list(txts.keys())
# ['9843',
# '7432',
# '2512',
# '3156',
# ...

#list(docs.items())
#[('9843',
#  Cuando pienso sobre una persona especial en mi vida, ...),
# ('7432',
#  Mi novia es muy importante en mi vida. Ella es muy ...),
# ('2512',
# ...

#type(list(docs.values())[0])
# spacy.tokens.doc.Doc

In [21]:
for key in set(txts).intersection(set(txts_test)):
    if key in ['5034']:
        continue
    print(key)
    assert txts[key] == txts_test[key]

5076
4523
4951
4626
5113
5321
5306
5169
5329
4736
4962
5320
2697
5021
5275
3291
3527
4375
5030
5161
4825
4473
4958
5224
3375
3986
5056
1942
4916
4655
4552
5397
4907
5340
4450
4884
5101
5287
5244
4607
5004
4515
4285
5053
4323
4388
5245
3863
4434
4325
5203
4348
4748
3407
3354
3740
4219
2068
4541
5353
4272
4793
5391
5371
2339
4455
5310
4987
5090
4215
5235
4922
4934
3159
5095
4411
5230
5234
4889
4945
4622
5312
4906
5335
5159
5128
4860
5326
4852
4502
5045
4600
4319
5211
4940
4653
4661
3886
3572
5345
5251
4715
5102
3342
5404
4560
5033
3383
4431
4759
4810
4608
4666
3925
5031
3960
5398
5314
5249
4447
4876
5044
5156
5387
4472
5085
5020
4920
4317
4953
5149
4519
3989
3950
4251
3678
5065
4800
2836
5097
3728
4888
4077
5166
5394
5115
4509
4694
4872
4965
5274
5195
5187
4952
4152
4689
5355
4773
5150
4576
4526
4514
4938
3541
4559
3877
3813
5077
5378
5277
5229
5280
3498
2118
5343
4281
5068
4465
4755
4564
4445
4767
4745
5262
4913
4024
2584
4750
4154
5311
4359
4879
4983
5257
5384
4606
4874
4019
4667
3926


In [174]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from langlearn.dataprep.transformers import SpacyDocsPreprocessor

spacy_wf = Pipeline([
  ('wordform', SpacyDocsPreprocessor(docs=docs)),
  ("count", CountVectorizer(
    tokenizer=lambda x: x,
    token_pattern=None,
    preprocessor=lambda x: [tok.strip() for tok in x],
    lowercase=False,
    analyzer='word',
    ngram_range=(1, 3),
    min_df = 5,
    #min_df = 0.15,
    # max_features=500
  ))
], verbose=True)

spacy_lem = Pipeline([
  ('lemma', SpacyDocsPreprocessor(docs=docs, token_attribute='lemma_')),
  ("count", CountVectorizer(
    tokenizer=lambda x: x,
    token_pattern=None,
    preprocessor=lambda x: x,
    lowercase=False,
    analyzer='word',
    ngram_range=(1, 3),
    min_df = 5,
    #min_df = 0.2,
    #max_features=500
  ))
], verbose=True)

spacy_pos = Pipeline([
  ('pos', SpacyDocsPreprocessor(docs=docs, token_attribute='pos_')),
  ("count", CountVectorizer(
    tokenizer=lambda x: x,
    token_pattern=None,
    preprocessor=lambda x: x,
    lowercase=False,
    analyzer='word',
    ngram_range=(1, 3),
    min_df = 5,
    #min_df = 0.15,
    # max_features=500
  ))
], verbose=True)

spacy_dep = Pipeline([
  ('dep', SpacyDocsPreprocessor(docs=docs, token_attribute=None)),
  ("count", CountVectorizer(
    tokenizer=lambda x: x,
    token_pattern=None,
    preprocessor=lambda x: [f"{tok.dep_} {tok.head.dep_}" for tok in x],
    lowercase=False,
    analyzer='word',
    ngram_range=(1, 1),
  ))
], verbose=True)

spacy_morph = Pipeline([
  ('morph', SpacyDocsPreprocessor(docs=docs, token_attribute='morph')),
  ("count", CountVectorizer(
    token_pattern=r"\b[,=\w]+\b",
    preprocessor=lambda x: str([' '.join(str(tok).split('|')) for tok in x]),
    lowercase=False,
    analyzer='word',
    ngram_range=(1, 2),
  ))
], verbose=True)

#res = spacy_wf.fit_transform(docs.keys())
#for feature in spacy_wf.get_feature_names_out()[0:150]:
#  print(feature)

In [6]:
print(df)
docs['4355']

     author essay  sequence_abs
0    A_3375  4355             0
1    A_3375  3375             1
2    A_3375  4466             2
3    A_3375  5217             3
4    A_3375  5310             4
..      ...   ...           ...
834  A_4314  4613             7
835  A_4314  4314             8
836  A_4131  4216             7
837  A_4131  4850             9
838  A_4131  4131             9

[839 rows x 3 columns]


Ciao!!!
Il mio cantante preferito si chiama Justin Bibler , ma non ho mai visto un suo concerto dal vivo. 
Mi chiamo Gaia, ho undici anni compiuti il primo Ottobre e frequento la prima media della scuola “Sant’Agata dei Goti”. Per quattro anni, non abitando nel “Centro storico” di Roma ho frequentato la scuola “Europa” nel plesso “Carlo Alberto Dalla Chiesa”. Mentre l’ultimo anno della scuola primaria sono stata nella scuola “Luigi Settembrini” del plesso “E. Q. Visconti”. Credo di essere molto preparata per poi affrontare ogni tipo di esame nella scuola secondaria. Da grande, vorrei diventare una ginecologa laureandomi in medicina dopo cinque anni del liceo classico. Mi piace molto la scuola perché ci si socializza con persone nuove e si imparano molte cose istruttive. La mia materia preferita è l’italiano, la storia e la scienze. Adoro lo sport, in particolare il nuoto e la pallavvolo. Come attività extrascolastiche pratico il nuoto il lunedì, martedì e venerdì. Il giovedì vado con l

In [175]:
df_full = pd.merge(df, df_malt, how='outer', on='essay')
df_full['author'] = df_full["author"].astype('category')
#df_full['essay'] = df_full["essay"].astype('category')

y = df_full['sequence_abs'].copy()
#y = y.astype(pd.CategoricalDtype(ordered=True))
#y = y.astype(int)
X = df_full.drop(columns='sequence_abs').copy()

#df_full

print(df_full[df_full.isna().any(axis=1)])
print(df_full[df_full.isnull().any(axis=1)])

df_full

Empty DataFrame
Columns: [author, essay, sequence_abs, predicted, probability_B1, probability_B2, probability_C1, probability_C2, scalarFeatures_Depth of the Parse Trees_0, scalarFeatures_Lexical Diversity_0, scalarFeatures_Maximal Non-Verbal Phrase - Mean_0, scalarFeatures_Maximal Non-Verbal Phrase - Std.Dev._0, scalarFeatures_Mean Length of Clauses (Tokens) - Mean_0, scalarFeatures_Non-Verbal Chains Length - Mean_0, scalarFeatures_Non-Verbal Chains Length - Std.Dev._0, scalarFeatures_Referential Cohesion - Mean_0, scalarFeatures_Referential Cohesion - Std.Dev._0, scalarFeatures_Sentence Length (in Tokens) - Mean_0, scalarFeatures_Sentence Length (in Tokens) - Std.Dev._0, scalarFeatures_Subordinate Ratio - Mean_0, scalarFeatures_Subordinate Ratio - Std.Dev._0, scalarFeatures_Text Length (in Lemmas)_0, scalarFeatures_Text Length (in Sentences)_0, scalarFeatures_Token Length - Mean_0, scalarFeatures_Token Length - Std.Dev._0, scalarFeatures_Verbal Roots_0, simpleScalarFeatures_Lunghezza

,author,essay,sequence_abs,predicted,probability_B1,probability_B2,probability_C1,probability_C2,scalarFeatures_Depth of the Parse Trees_0,scalarFeatures_Lexical Diversity_0,...,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Astrattezza_0,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Semiastrattezza_0,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Concretezza_0,simpleDistrFeatures_Sofisticatezza lessicale_parole vuote_0,simpleDistrFeatures_Sofisticatezza lessicale_lemmi vuoti_0,simpleDistrFeatures_Sofisticatezza lessicale_parole contenuto_0,simpleDistrFeatures_Sofisticatezza lessicale_lemmi contenuto_0,simpleDistrFeatures_Tasso di vocabolario di base_Fondamentali_0,simpleDistrFeatures_Tasso di vocabolario di base_Alto uso_0,simpleDistrFeatures_Tasso di vocabolario di base_Alto disponibilità_0
0,A_3375,4355,0,1,0.506112,0.349380,0.144508,0.0,9.0,0.402510,...,0.025316,0.316456,0.658228,0.013592,0.015009,0.000429,0.000413,0.854962,0.114504,0.030534
1,A_3375,3375,1,2,0.325794,0.511743,0.162462,0.0,10.0,0.383010,...,0.116279,0.372093,0.511628,0.009919,0.010709,0.000845,0.000645,0.919118,0.066176,0.014706
2,A_3375,4466,2,2,0.347513,0.491908,0.160579,0.0,8.0,0.376060,...,0.181818,0.127273,0.690909,0.013394,0.011012,0.000554,0.000700,0.846774,0.096774,0.056452
3,A_3375,5217,3,1,0.494528,0.348384,0.157088,0.0,10.0,0.392630,...,0.097561,0.365854,0.536585,0.010912,0.011711,0.000835,0.000794,0.940678,0.050847,0.008475
4,A_3375,5310,4,2,0.182036,0.484657,0.333307,0.0,8.0,0.403300,...,0.149254,0.447761,0.402985,0.012638,0.010282,0.000472,0.000573,0.896104,0.090909,0.012987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,A_4314,4613,7,3,0.029305,0.463089,0.507606,0.0,11.0,0.417210,...,0.072917,0.375000,0.552083,0.011855,0.009249,0.000600,0.000498,0.923077,0.060440,0.016484
835,A_4314,4314,8,1,0.516822,0.327856,0.155322,0.0,15.0,0.436890,...,0.069364,0.213873,0.716763,0.014033,0.008797,0.000571,0.000346,0.817460,0.150794,0.031746
836,A_4131,4216,7,1,0.517918,0.327224,0.154858,0.0,13.0,0.385330,...,0.050000,0.150000,0.800000,0.014866,0.013510,0.000454,0.000552,0.880435,0.076087,0.043478
837,A_4131,4131,9,1,0.518043,0.327208,0.154750,0.0,5.0,0.378947,...,0.000000,0.666667,0.333333,0.009725,0.015770,0.000228,0.000608,0.977778,0.000000,0.022222


In [199]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn import svm
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate, cross_val_predict, GroupKFold
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.tree import DecisionTreeRegressor

from langlearn.dataprep.pipes import sklearn_pipe
from langlearn.dataprep.transformers import GroupNormalizer

#df_full_scaled = gn.transform(df_full.iloc[0:100, :])
#df_full_scaled = gn.transform(df_full)

pipe = sklearn_pipe(X, verbose=True)
spacy_fu = FeatureUnion(transformer_list=[
        ('spacy_wf', spacy_wf),
        #('spacy_lem', spacy_wf),
        ('spacy_pos', spacy_pos),
        ('spacy_dep', spacy_dep),
        ('spacy_morph', spacy_morph),
    ], verbose=True)

spacy_pipe = ColumnTransformer([
        ('spacy_pipe', spacy_fu, 'essay'),
    ],
    remainder='drop',
    n_jobs=1, 
    verbose=True
)

gn_pipe = Pipeline([
    ('fu', FeatureUnion([
        ('ct_passthrough_author', ColumnTransformer([
            ('passthrough', 'passthrough', ['author'])
            ], remainder='drop', verbose=True)
        ),
        ('ct_passthrough_numbers', ColumnTransformer([
                ('passthrough', 'passthrough', make_column_selector(dtype_include='number')),
            ], remainder='drop', verbose=True)
        ),
        ], verbose=True)
    ),
    ('gn', GroupNormalizer())
], verbose=True).set_output(transform='pandas')

combined_features = FeatureUnion([
    ("pipe", pipe),
    ('gn', gn_pipe),
    ('spacy', spacy_pipe),
    ],
    verbose=True
)

scoring = ['neg_mean_absolute_error', 'explained_variance', 'r2']

Categorical columns are: ['author']
Ordinal columns are: ['predicted']
Numerical columns are: ['probability_B1', 'probability_B2', 'probability_C1', 'probability_C2', 'scalarFeatures_Depth of the Parse Trees_0', 'scalarFeatures_Lexical Diversity_0', 'scalarFeatures_Maximal Non-Verbal Phrase - Mean_0', 'scalarFeatures_Maximal Non-Verbal Phrase - Std.Dev._0', 'scalarFeatures_Mean Length of Clauses (Tokens) - Mean_0', 'scalarFeatures_Non-Verbal Chains Length - Mean_0', 'scalarFeatures_Non-Verbal Chains Length - Std.Dev._0', 'scalarFeatures_Referential Cohesion - Mean_0', 'scalarFeatures_Referential Cohesion - Std.Dev._0', 'scalarFeatures_Sentence Length (in Tokens) - Mean_0', 'scalarFeatures_Sentence Length (in Tokens) - Std.Dev._0', 'scalarFeatures_Subordinate Ratio - Mean_0', 'scalarFeatures_Subordinate Ratio - Std.Dev._0', 'scalarFeatures_Text Length (in Lemmas)_0', 'scalarFeatures_Text Length (in Sentences)_0', 'scalarFeatures_Token Length - Mean_0', 'scalarFeatures_Token Length - Std

In [200]:
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import FunctionTransformer
from langlearn.dataprep.transformers import GroupNormalizer

stdscaler = StandardScaler() # with_mean=False)

#pca = PCA(100)
pca = TruncatedSVD(n_components=125)

dtregressor = DecisionTreeRegressor(random_state=42)
hgbreg = HistGradientBoostingRegressor(loss='squared_error')
svmreg = svm.SVR(kernel="rbf", C=1e1, gamma=0.1) #, C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
krreg = KernelRidge(kernel='rbf')


estimators = [svmreg, hgbreg, krreg]

In [207]:
#X_in = X.iloc[0:15, :]
#y_in = y[0:15]
#
X_in = X.copy()
y_in = y.copy()

proc_pipe = Pipeline(steps=[
    ('combined_features', combined_features),
    ('to_dense', FunctionTransformer(lambda x: np.asarray(x.todense()), accept_sparse=True)),
    ('scaler_std', stdscaler),
    ('redux_pca', pca),
    ('estimator_svmreg', estimators[1])
], verbose=True)

y_in

0      0
1      1
2      2
3      3
4      4
      ..
834    7
835    8
836    7
837    9
838    9
Name: sequence_abs, Length: 839, dtype: int64

In [43]:
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline

scores = cross_validate(proc_pipe, X_in, y_in, cv=KFold(n_splits=3, shuffle=True, random_state=24), scoring=scoring, n_jobs=1, verbose=True, error_score='raise')
print(scores)
for score in scores:
    print(f"{score}: {scores[score].mean()} {scores[score].std()}")

[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (1 of 3) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (2 of 3) Processing ord, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing imputer, total=   0.0s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s
[FeatureUnion] .......... (step 1 of 1) Processing pipe, total=   0.0s
[Pipeline] . (step 1 of 2) Processing combined_features, total=   0.0s
[Pipeline] .. (step 2 of 2) Processing estimator_svmreg, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (1 of 3) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (2 of 3) Processing ord, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing imputer, total=   0.0s
[Colum

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s finished


In [202]:
import sklearn.metrics
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

X_in = X.copy()
y_in = y.copy()

for estimator in estimators[1:2]:

    for processor, preprocessor_name, _X, _y in [(proc_pipe, 'preproc_pipe', X_in, y_in)]:
        print(f'{estimator} for {preprocessor_name}')
        
        # scores = cross_validate(make_pipeline(preprocessor, estimator), X, y, return_estimator=True, cv=10, scoring=scoring, n_jobs=-1)
        scores = cross_validate(processor, _X.copy(), _y.copy(), return_estimator=False, groups=_X['author'], 
                                cv=GroupKFold(n_splits=3), scoring=scoring, n_jobs=1, error_score='raise', verbose=True)
        print(scores)
        score_maxlength = max([len(score) for score in scores if score.startswith('test_')])
        for score in [score for score in scores if score.startswith('test_')]:
            print(f"{score: <{score_maxlength}}: {scores[score].mean(): .4f} {scores[score].std(): .4f}")
                
        #y_preds = list()
        #for trained_esimator in scores['estimator']:
        #    y_preds.extend(cross_val_predict(trained_esimator, X, y, groups=X['author'], cv=GroupKFold(n_splits=10), n_jobs=-1))
        y_pred = cross_val_predict(processor, _X.copy(), _y.copy(), groups=_X['author'], 
                                   cv=GroupKFold(n_splits=3), n_jobs=1, verbose=True)
        #print(y_pred)
        # print(np.array(y_preds).round(0).astype(int))
        print(sklearn.metrics.confusion_matrix(_y, y_pred.round(0).astype(int)))

        import matplotlib.pyplot as plt
        from sklearn.metrics import PredictionErrorDisplay

        fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
        PredictionErrorDisplay.from_predictions(
            _y,
            y_pred=y_pred,
            kind="actual_vs_predicted",
            #subsample=250,
            ax=axs[0],
            random_state=0,
        )
        axs[0].set_title("Actual vs. Predicted values")
        
        PredictionErrorDisplay.from_predictions(
            _y,
            y_pred=y_pred,
            kind="residual_vs_predicted",
            #subsample=250,
            ax=axs[1],
            random_state=0,
        )
        axs[1].set_title("Residuals vs. Predicted Values")
        
        fig.suptitle("Plotting cross-validated predictions")
        plt.tight_layout()
        plt.show()
        
        print('####\n')
        # https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html

HistGradientBoostingRegressor() for preproc_pipe


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (1 of 3) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (2 of 3) Processing ord, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s
[FeatureUnion] .......... (step 1 of 3) Processing pipe, total=   0.0s
[ColumnTransformer] ... (1 of 1) Processing passthrough, total=   0.0s
[FeatureUnion]  (step 1 of 2) Processing ct_passthrough_author, total=   0.0s
[ColumnTransformer] ... (1 of 1) Processing passthrough, total=   0.0s
[FeatureUnion]  (step 2 of 2) Processing ct_passthrough_numbers, total=   0.0s
[Pipeline] ................ (step 1 of 2) Processing fu, total=   0.0s
GN.fit: (559, 156)
ct_passthrough_author__passthrough__author


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  6.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'fit_time': array([92.35137105, 73.35504913, 28.14166784]), 'score_time': array([1.41998029, 1.81187057, 0.9666121 ]), 'test_neg_mean_absolute_error': array([-2.21135849, -2.36806676, -2.37939709]), 'test_explained_variance': array([0.34489916, 0.24752218, 0.15673057]), 'test_r2': array([0.33991347, 0.24109432, 0.15638299])}
test_neg_mean_absolute_error: -2.3196  0.0767
test_explained_variance     :  0.2497  0.0768
test_r2                     :  0.2458  0.0750


KeyboardInterrupt: 

In [32]:
#local_pipe.score(X_in, y_in)

0.4708780135319408

# Predict

In [203]:
testing_cita_tsv = data_prefix+'/raw/LangLearn_Test_Data/CItA/Test_CItA.tsv'
df_test,tsv_test = read_cita_tsv(testing_cita_tsv)
df_test

,author,essay,sequence_abs
0,A_3116,5388,0
1,A_3116,5222,0
2,A_3116,4853,0
3,A_3116,3116,0
4,A_3116,5192,0
...,...,...,...
455,A_4019,4019,7
456,A_4152,5201,1
457,A_4152,4152,7
458,A_3291,3291,1


In [206]:
from collections import defaultdict

import numpy as np
import pandas as pd


dtypes = defaultdict(np.float32, fn='str', predicted='category')

testing_cita_malt = data_prefix + '/interim/LangLearn_Test_Data/CItA/malt.csv'

df_malt_test = pd.read_csv(testing_cita_malt, sep=',', dtype=dtypes, keep_default_na=False)
df_malt_test['predicted'].replace(['B1', 'B2', 'C1', 'C2'], [1,2,3,4], inplace=True)
df_malt_test['predicted'] = df_malt_test['predicted'].astype(pd.CategoricalDtype(ordered=True))
df_malt_test.rename(columns={"fn":"essay"}, inplace=True)
df_malt_test['essay'] = df_malt_test['essay'].str.replace('.txt','')
df_malt_test.sort_values('essay', inplace=True, ignore_index=True)

df_malt_test_numbers = df_malt_test.select_dtypes('number').copy()
df_malt_test.drop(columns=df_malt_test_numbers.columns, inplace=True)
df_malt_test = pd.concat([df_malt_test, df_malt_vt.transform(df_malt_test_numbers)], axis=1)

print(df_malt_test[df_malt_test.isna().any(axis=1)])
print(df_malt_test[df_malt_test.isnull().any(axis=1)])
print("#")
print(df_malt_test)

Empty DataFrame
Columns: [essay, predicted, probability_B1, probability_B2, probability_C1, probability_C2, scalarFeatures_Depth of the Parse Trees_0, scalarFeatures_Lexical Diversity_0, scalarFeatures_Maximal Non-Verbal Phrase - Mean_0, scalarFeatures_Maximal Non-Verbal Phrase - Std.Dev._0, scalarFeatures_Mean Length of Clauses (Tokens) - Mean_0, scalarFeatures_Non-Verbal Chains Length - Mean_0, scalarFeatures_Non-Verbal Chains Length - Std.Dev._0, scalarFeatures_Referential Cohesion - Mean_0, scalarFeatures_Referential Cohesion - Std.Dev._0, scalarFeatures_Sentence Length (in Tokens) - Mean_0, scalarFeatures_Sentence Length (in Tokens) - Std.Dev._0, scalarFeatures_Subordinate Ratio - Mean_0, scalarFeatures_Subordinate Ratio - Std.Dev._0, scalarFeatures_Text Length (in Lemmas)_0, scalarFeatures_Text Length (in Sentences)_0, scalarFeatures_Token Length - Mean_0, scalarFeatures_Token Length - Std.Dev._0, scalarFeatures_Verbal Roots_0, simpleScalarFeatures_Lunghezza del testo_0, simpleSc

In [205]:
#df_ctap_test['essay']
# Name: essay, Length: 468, dtype: objec
#df_malt_test['essay']
# Name: essay, Length: 471, dtype: object

In [120]:
#import spacy
#nlp = spacy.load("it_core_news_lg")

import langlearn.dataprep.utils
training_cita_essays_fn = data_prefix + '/interim/LangLearn_Test_Data/CItA/Essays_Test_CItA.xml'

txts_test = langlearn.dataprep.utils.get_txts(testing_cita_essays_fn, shrink_whitespaces=True)
docs.update(dict(zip(list(txts_test.keys()), list(nlp.pipe(list(txts_test.values()))))))

In [121]:
docs['1942']

Un momento di sconforto
Un pomeriggio caldo, come gli altri, Luigino stava nella sua cameretta a vedere la TV con il suo cane Willy. Decide di andare in cucina a prendere una bottiglietta d’acqua e la poggiò sul tavolo, e l’acqua si muoveva, si muoveva tutto!Era un terremoto, Willy spaventato va da Luigino cercarono di uscire ma c’era qualcosa che bloccava la porta, la madre era riuscita a scappare ma non trovava Luigino. Lui, ormai stanco di provare ad aprire la porta, si nascose sotto al tavolo con il suo cane. Luigino perse i sensi e sviene . Quando si risveglia era coperto di polvere e macerie ma non riusciva a capire cosa era successo, ma poi, dopo tante domande che si stava facendo si ricordò dell’accaduto. Era molto preoccupato sia per lui che per il suo cagnolino, non sapeva cosa fare, cercava di togliere un po di cose su di lui ma niente. Sentiva il rumore delle scavatrici e della gente che urlava, ma lui era troppo stanco e senza accorgersene si addormentò. Ormai erano due gi

In [210]:
df_full_test = pd.merge(df_test, df_malt_test, how='outer', on='essay')
df_full_test['author'] = df_full_test["author"].astype('category')
#df_full['essay'] = df_full["essay"].astype('category')

X_test = df_full_test.drop(columns='sequence_abs')
#df_full

print(df_full_test[df_full_test.isna().any(axis=1)])
print(df_full_test[df_full_test.isnull().any(axis=1)])

X_test

Empty DataFrame
Columns: [author, essay, sequence_abs, predicted, probability_B1, probability_B2, probability_C1, probability_C2, scalarFeatures_Depth of the Parse Trees_0, scalarFeatures_Lexical Diversity_0, scalarFeatures_Maximal Non-Verbal Phrase - Mean_0, scalarFeatures_Maximal Non-Verbal Phrase - Std.Dev._0, scalarFeatures_Mean Length of Clauses (Tokens) - Mean_0, scalarFeatures_Non-Verbal Chains Length - Mean_0, scalarFeatures_Non-Verbal Chains Length - Std.Dev._0, scalarFeatures_Referential Cohesion - Mean_0, scalarFeatures_Referential Cohesion - Std.Dev._0, scalarFeatures_Sentence Length (in Tokens) - Mean_0, scalarFeatures_Sentence Length (in Tokens) - Std.Dev._0, scalarFeatures_Subordinate Ratio - Mean_0, scalarFeatures_Subordinate Ratio - Std.Dev._0, scalarFeatures_Text Length (in Lemmas)_0, scalarFeatures_Text Length (in Sentences)_0, scalarFeatures_Token Length - Mean_0, scalarFeatures_Token Length - Std.Dev._0, scalarFeatures_Verbal Roots_0, simpleScalarFeatures_Lunghezza

,author,essay,predicted,probability_B1,probability_B2,probability_C1,probability_C2,scalarFeatures_Depth of the Parse Trees_0,scalarFeatures_Lexical Diversity_0,scalarFeatures_Maximal Non-Verbal Phrase - Mean_0,...,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Astrattezza_0,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Semiastrattezza_0,simpleDistrFeatures_Distribuzione dell'astrattezza/concretezza_Concretezza_0,simpleDistrFeatures_Sofisticatezza lessicale_parole vuote_0,simpleDistrFeatures_Sofisticatezza lessicale_lemmi vuoti_0,simpleDistrFeatures_Sofisticatezza lessicale_parole contenuto_0,simpleDistrFeatures_Sofisticatezza lessicale_lemmi contenuto_0,simpleDistrFeatures_Tasso di vocabolario di base_Fondamentali_0,simpleDistrFeatures_Tasso di vocabolario di base_Alto uso_0,simpleDistrFeatures_Tasso di vocabolario di base_Alto disponibilità_0
0,A_3116,5388,1,0.519096,0.328663,0.152240,0.0,7.0,0.38085,2.913043,...,0.041667,0.166667,0.791667,0.010394,0.014253,0.000835,0.000778,0.911765,0.058824,0.029412
1,A_3116,5388,1,0.519096,0.328663,0.152240,0.0,7.0,0.38085,2.913043,...,0.041667,0.166667,0.791667,0.010394,0.014253,0.000835,0.000778,0.911765,0.058824,0.029412
2,A_3116,5388,1,0.519096,0.328663,0.152240,0.0,7.0,0.38085,2.913043,...,0.041667,0.166667,0.791667,0.010394,0.014253,0.000835,0.000778,0.911765,0.058824,0.029412
3,A_3116,5388,1,0.519096,0.328663,0.152240,0.0,7.0,0.38085,2.913043,...,0.041667,0.166667,0.791667,0.010394,0.014253,0.000835,0.000778,0.911765,0.058824,0.029412
4,A_3116,5388,1,0.519096,0.328663,0.152240,0.0,7.0,0.38085,2.913043,...,0.041667,0.166667,0.791667,0.010394,0.014253,0.000835,0.000778,0.911765,0.058824,0.029412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,A_4019,4019,1,0.515587,0.328727,0.155686,0.0,8.0,0.33522,6.571429,...,0.545455,0.181818,0.272727,0.009242,0.012187,0.000797,0.000969,0.932432,0.067568,0.000000
456,A_4152,5201,2,0.322300,0.515729,0.161972,0.0,10.0,0.37270,9.652174,...,0.203390,0.389830,0.406780,0.010733,0.009603,0.000755,0.000615,0.923077,0.070513,0.006410
457,A_4152,4152,1,0.490328,0.350545,0.159126,0.0,7.0,0.39547,6.160000,...,0.117647,0.235294,0.647059,0.014757,0.010823,0.000696,0.000420,0.853147,0.132867,0.013986
458,A_3291,3291,1,0.497829,0.351079,0.151092,0.0,12.0,0.38645,6.000000,...,0.050000,0.133333,0.816667,0.013874,0.010843,0.000568,0.000596,0.842975,0.132231,0.024793


In [213]:
proc_pipe.fit(X_in, y_in)

y_pred = proc_pipe.predict(X_test)

[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (1 of 3) Processing cat, total=   0.0s
[Pipeline] ........... (step 1 of 1) Processing encoder, total=   0.0s
[ColumnTransformer] ........... (2 of 3) Processing ord, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s
[FeatureUnion] .......... (step 1 of 3) Processing pipe, total=   0.0s
[ColumnTransformer] ... (1 of 1) Processing passthrough, total=   0.0s
[FeatureUnion]  (step 1 of 2) Processing ct_passthrough_author, total=   0.0s
[ColumnTransformer] ... (1 of 1) Processing passthrough, total=   0.0s
[FeatureUnion]  (step 2 of 2) Processing ct_passthrough_numbers, total=   0.0s
[Pipeline] ................ (step 1 of 2) Processing fu, total=   0.0s
GN.fit: (839, 156)
GN.transform: (839, 156)
...GN.transformed:

In [214]:
y_pred

array([ 7.27764026,  7.27764026,  7.27764026,  7.27764026,  7.27764026,
        7.27764026,  4.83292432,  4.83292432,  4.83292432,  4.83292432,
        4.97705641,  4.97705641,  4.97705641,  5.10217324,  5.10217324,
        5.10217324,  5.10217324,  1.70614891,  1.70614891,  1.70614891,
        1.70614891,  9.4850083 ,  9.4850083 ,  9.4850083 ,  9.4850083 ,
        9.4850083 ,  9.4850083 ,  6.37185948,  6.37185948,  6.53852229,
        6.53852229,  6.53852229,  6.53852229,  4.42664838,  4.42664838,
        4.42664838,  5.88168301,  5.88168301,  5.88168301,  5.18700022,
        5.18700022,  5.18700022,  5.18700022,  5.18700022,  6.76612744,
        6.76612744,  6.76612744,  6.76612744,  6.40583384,  6.40583384,
        6.40583384,  6.40583384,  6.40583384,  1.14051083,  1.14051083,
        1.14051083,  1.14051083,  4.60640003,  4.60640003,  4.60640003,
        4.60640003,  4.60640003,  4.62775853,  4.62775853,  4.62775853,
        3.29418105,  3.29418105,  3.29418105,  3.29418105,  7.19

In [215]:
tsv_test = pd.read_csv(testing_cita_tsv, sep='\t', dtype=str)
tsv_test['Pred'] = -1
for row_id, row in tsv_test.iterrows():
    essay_1 = row['Essay_1']
    essay_2 = row['Essay_2']
    essay_1_pred = np.average(y_pred[X_test[X_test['essay'] == essay_1].index])
    essay_2_pred = np.average(y_pred[X_test[X_test['essay'] == essay_2].index])
    label = 0 if essay_1_pred <= essay_2_pred else 1
    print(essay_1,essay_1_pred,essay_2,essay_2_pred,label)
    tsv_test.loc[row_id, 'Pred'] = str(label)

tsv_test.to_csv('cita-preds.tsv', sep='\t', index=False)

5388 7.277640258631695 4853 4.977056407979215 1
4614 5.187000215174842 5358 6.76612744401976 0
5144 7.123278494874202 3364 4.930590878034086 1
4749 4.4266483809151325 4814 6.5385222924788176 0
5358 6.76612744401976 3833 5.88168300586769 1
5162 3.763833175101253 3364 4.930590878034086 0
5146 5.953205435951152 5400 3.2746277904089798 1
5144 7.123278494874202 5173 6.468691710547123 1
5380 3.4171504724099067 5400 3.2746277904089798 1
5359 3.5926745248979244 5299 2.5677748955095625 1
5319 6.3718594817430265 4749 4.4266483809151325 1
4181 6.405833836620728 5358 6.76612744401976 0
4853 4.977056407979215 5222 4.832924322358846 1
5223 5.385135395139522 4258 4.949547111714538 1
5192 1.7061489082798302 4853 4.977056407979215 0
5091 6.277942926435309 5144 7.123278494874202 0
4047 4.627758530364164 3833 5.88168300586769 0
5400 3.2746277904089798 3794 6.9691631444001985 0
5359 3.5926745248979244 4716 7.467733763442195 0
5192 1.7061489082798302 3116 5.102173235944682 0
4258 4.949547111714538 5075 4.0

In [66]:
#X_test[X_test['essay'] == '1066']
df_test[df_test['essay'] == '1066']

,author,essay,sequence_abs
4,A_1066,1066,6
7,A_1066,1066,7
10,A_1066,1066,8
